# Experient imputation methods with AIF360

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils.completer import complete_by_mean_col, complete_by_mean_col_v2
from utils.completer import complete_by_multi, complete_by_multi_v2
from utils.completer import complete_by_similar_row, complete_by_similar_row_v2
from utils.generator import gen_complete_random
from utils.data import create_adult_dataset

### Learning Fair Representations (LFR)
[example notebook](https://github.com/Trusted-AI/AIF360/blob/master/examples/demo_lfr.ipynb)  

------

The idea is to first apply imputation on original dataset (adult dataset here)  
Then feed the converted dataset into LFR to see any difference  

------

Note:  
In order to LFR to work on Python3.8  
Should install LFR directly from github, instead of from pip  

In [2]:
from aif360.datasets import BinaryLabelDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.lfr import LFR

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [4]:
data = create_adult_dataset()
print(data.X.shape)
print(data.X.isnull().sum().sum())

(32561, 10)
0


In [5]:
data_incomplete = gen_complete_random(data, random_ratio=0.2)
print(data_incomplete.X.isnull().sum())

gen_complete_random: 58244 NaN values have been inserted
age               6435
workclass         6423
education         6525
education-num     6467
marital-status    6544
occupation        6499
relationship      6471
race              6383
hours-per-week    6497
sex                  0
dtype: int64


In [6]:
data_complete = complete_by_mean_col(data_incomplete)
print(data_complete.X.isnull().sum())

age               0
workclass         0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
hours-per-week    0
sex               0
dtype: int64


In [7]:
data_complete.y_encoder.classes_

array([' <=50K', ' >50K'], dtype=object)

In [8]:
data_complete.categorical_features

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race']

In [9]:
data_complete.protected_features

['sex']

In [10]:
# convert to standard dataset
from sklearn.model_selection import train_test_split
tmp_df = pd.concat([data_complete.X, pd.DataFrame(data_complete.y, columns=["_TARGET_"])], axis=1).copy()
tmp_df['sex'] = tmp_df['sex'].apply(lambda x : 0.0 if x == ' Female' else 1.0) # necessary for aif360 to understand
tmp_df_train, tmp_df_test = train_test_split(tmp_df, test_size=0.3, shuffle=True)
data_imputed_train = StandardDataset(df=tmp_df_train, label_name="_TARGET_", favorable_classes=lambda x: x > 0,
                                     protected_attribute_names=['sex'], privileged_classes=[[1.0]],
                                     features_to_keep=data_complete.X.columns.tolist(),
                                     instance_weights_name=None, features_to_drop=[],
                                     custom_preprocessing=None, categorical_features=data_complete.categorical_features)
data_imputed_test = StandardDataset(df=tmp_df_test, label_name="_TARGET_", favorable_classes=lambda x: x > 0,
                                    protected_attribute_names=['sex'], privileged_classes=[[1.0]],
                                    features_to_keep=data_complete.X.columns.tolist(),
                                    instance_weights_name=None, features_to_drop=[],
                                    custom_preprocessing=None, categorical_features=data_complete.categorical_features)
print(tmp_df_train.shape)
print(tmp_df_test.shape)

(22792, 11)
(9769, 11)


In [11]:
privileged_groups = [{'sex': 1.0}]
unprivileged_groups = [{'sex': 0.0}]
metric_imputed_train = BinaryLabelDatasetMetric(data_imputed_train, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
metric_imputed_test = BinaryLabelDatasetMetric(data_imputed_test, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
print(metric_imputed_train.mean_difference())
print(metric_imputed_test.mean_difference())

-0.19845762081733645
-0.1911858703853891


In [12]:
scaler = StandardScaler()
data_imputed_train.features = scaler.fit_transform(data_imputed_train.features)
data_imputed_test.features = scaler.transform(data_imputed_test.features)

In [13]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0,
         verbose=1
        )
TR = TR.fit(data_imputed_train, maxiter=5000, maxfun=5000)

step: 0, loss: 0.9896145477518621, L_x: 2.5506503453342653,  L_y: 0.7247481625639295,  L_z: 0.004900675327253022
step: 250, loss: 0.9896145225098801, L_x: 2.5506503674769796,  L_y: 0.724748137237789,  L_z: 0.0049006742621965655
step: 500, loss: 0.9896145466012233, L_x: 2.550650371640526,  L_y: 0.7247481673870411,  L_z: 0.004900671025064818
step: 750, loss: 0.8871006796057351, L_x: 2.5495404005205486,  L_y: 0.6228296373355169,  L_z: 0.0046585011090816645
step: 1000, loss: 0.8871006870140217, L_x: 2.5495404069945944,  L_y: 0.6228296468793166,  L_z: 0.004658499717622798
step: 1250, loss: 0.8871006667299441, L_x: 2.5495404001849726,  L_y: 0.622829621281813,  L_z: 0.004658502714816913
step: 1500, loss: 0.8117905572462389, L_x: 2.5456845610942844,  L_y: 0.5484700846652261,  L_z: 0.0043760082357922215
step: 1750, loss: 0.8117905495159956, L_x: 2.545684568923784,  L_y: 0.5484700793038377,  L_z: 0.004376006659889686
step: 2000, loss: 0.8065109955517435, L_x: 2.543394127179285,  L_y: 0.544591677

In [15]:
data_imputed_transf_train = TR.transform(data_imputed_train)
data_imputed_transf_test = TR.transform(data_imputed_test)
print(classification_report(data_imputed_test.labels, data_imputed_transf_test.labels, zero_division=0))
metric_imputed_transf_train = BinaryLabelDatasetMetric(data_imputed_transf_train, 
                                                       unprivileged_groups=unprivileged_groups,
                                                       privileged_groups=privileged_groups)
metric_imputed_transf_test = BinaryLabelDatasetMetric(data_imputed_transf_test, 
                                                      unprivileged_groups=unprivileged_groups,
                                                      privileged_groups=privileged_groups)
print(metric_imputed_transf_train.mean_difference())
print(metric_imputed_transf_test.mean_difference())

              precision    recall  f1-score   support

         0.0       0.76      1.00      0.86      7432
         1.0       0.00      0.00      0.00      2337

    accuracy                           0.76      9769
   macro avg       0.38      0.50      0.43      9769
weighted avg       0.58      0.76      0.66      9769

0.0
0.0


In [16]:
# compare with original dataset
tmp_df = pd.concat([data.X, pd.DataFrame(data.y, columns=["_TARGET_"])], axis=1).copy()
tmp_df['sex'] = tmp_df['sex'].apply(lambda x : 0.0 if x == ' Female' else 1.0) # necessary for aif360 to understand
tmp_df_train, tmp_df_test = train_test_split(tmp_df, test_size=0.3, shuffle=True)
data_train = StandardDataset(df=tmp_df_train, label_name="_TARGET_", favorable_classes=lambda x: x > 0,
                             protected_attribute_names=['sex'], privileged_classes=[[1.0]],
                             features_to_keep=data_complete.X.columns.tolist(),
                             instance_weights_name=None, features_to_drop=[],
                             custom_preprocessing=None, categorical_features=data_complete.categorical_features)
data_test = StandardDataset(df=tmp_df_test, label_name="_TARGET_", favorable_classes=lambda x: x > 0,
                            protected_attribute_names=['sex'], privileged_classes=[[1.0]],
                            features_to_keep=data_complete.X.columns.tolist(),
                            instance_weights_name=None, features_to_drop=[],
                            custom_preprocessing=None, categorical_features=data_complete.categorical_features)
print(tmp_df_train.shape)
print(tmp_df_test.shape)
metric_train = BinaryLabelDatasetMetric(data_train, 
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)
metric_test = BinaryLabelDatasetMetric(data_test, 
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
print(metric_train.mean_difference())
print(metric_test.mean_difference())

(22792, 11)
(9769, 11)
-0.19347587452471865
-0.20266969567058415


In [19]:
data_train.features = scaler.fit_transform(data_train.features)
data_test.features = scaler.transform(data_test.features)
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0,
         verbose=1
        )
TR = TR.fit(data_train, maxiter=5000, maxfun=5000)

step: 0, loss: 0.9310241358214724, L_x: 2.5070259834670834,  L_y: 0.6630164435129302,  L_z: 0.008652546980916876
step: 250, loss: 0.9310241479293173, L_x: 2.5070260229606296,  L_y: 0.6630164577066221,  L_z: 0.008652543963316179
step: 500, loss: 0.9310241410569131, L_x: 2.5070259908939905,  L_y: 0.6630164383652646,  L_z: 0.008652551801124758
step: 750, loss: 0.8713837661082762, L_x: 2.506429235381934,  L_y: 0.6041444579964163,  L_z: 0.008298192286833256
step: 1000, loss: 0.871383764195668, L_x: 2.506429213277862,  L_y: 0.6041444603712717,  L_z: 0.008298191248305038
step: 1250, loss: 0.8713837705198791, L_x: 2.5064292281649134,  L_y: 0.6041444692179301,  L_z: 0.008298189242728864
step: 1500, loss: 0.8286751898569678, L_x: 2.5038802214802947,  L_y: 0.5647548948882384,  L_z: 0.006766136410349978
step: 1750, loss: 0.8286751987845065, L_x: 2.503880180373343,  L_y: 0.5647549103926711,  L_z: 0.006766135177250582
step: 2000, loss: 0.8230262234325005, L_x: 2.502949838919146,  L_y: 0.560618292585

In [20]:
data_transf_train = TR.transform(data_train)
data_transf_test = TR.transform(data_test)
print(classification_report(data_test.labels, data_transf_test.labels, zero_division=0))
metric_transf_train = BinaryLabelDatasetMetric(data_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
metric_transf_test = BinaryLabelDatasetMetric(data_transf_test, 
                                              unprivileged_groups=unprivileged_groups,
                                              privileged_groups=privileged_groups)
print(metric_transf_train.mean_difference())
print(metric_transf_test.mean_difference())

              precision    recall  f1-score   support

         0.0       0.76      1.00      0.87      7459
         1.0       0.00      0.00      0.00      2310

    accuracy                           0.76      9769
   macro avg       0.38      0.50      0.43      9769
weighted avg       0.58      0.76      0.66      9769

0.0
0.0
